In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from Functions import *
import folium

In [3]:
# Import and set up the data
df = pd.read_csv('NDT_dataset_clean_2.csv', index_col=0)
df

,idle_start_time,idle_end_time,idle_duration_hr,idle_location_x,idle_location_y,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt
0,2025-05-08 00:44:15+00:00,2025-05-08 00:55:48+00:00,0.192500,-1140674450,509021916,2025-05-07 18:44:15-06:00,2025-05-07 18:55:48-06:00,18.916667
1,2025-05-08 01:06:07+00:00,2025-05-08 02:03:30.010000+00:00,0.956392,-1140102166,508591116,2025-05-07 19:06:07-06:00,2025-05-07 20:03:30.010000-06:00,19.050000
2,2025-05-08 02:21:39+00:00,2025-05-08 02:27:57.010000+00:00,0.105003,-1141278866,509446300,2025-05-07 20:21:39-06:00,2025-05-07 20:27:57.010000-06:00,20.450000
3,2025-05-08 02:29:56.010000+00:00,2025-05-08 13:51:49+00:00,11.364719,-1141283733,509446566,2025-05-07 20:29:56.010000-06:00,2025-05-08 07:51:49-06:00,20.850000
4,2025-05-08 14:19:32.010000+00:00,2025-05-08 14:45:09+00:00,0.426942,-1141400783,510382683,2025-05-08 08:19:32.010000-06:00,2025-05-08 08:45:09-06:00,8.750000
...,...,...,...,...,...,...,...,...
313,2025-06-10 00:01:37+00:00,2025-06-10 01:13:04+00:00,1.190833,-1140877250,511574866,2025-06-09 18:01:37-06:00,2025-06-09 19:13:04-06:00,18.216667
314,2025-06-10 01:51:55+00:00,2025-06-10 14:28:37+00:00,12.611667,-1141283433,509446050,2025-06-09 19:51:55-06:00,2025-06-10 08:28:37-06:00,19.466667
315,2025-06-10 15:24:35+00:00,2025-06-10 16:20:23+00:00,0.930000,-1141091933,511507583,2025-06-10 09:24:35-06:00,2025-06-10 10:20:23-06:00,9.333333
316,2025-06-10 16:22:11+00:00,2025-06-10 16:31:29+00:00,0.155000,-1141093166,511512416,2025-06-10 10:22:11-06:00,2025-06-10 10:31:29-06:00,10.516667


## Preprocessing and Feature Engineering

In [5]:
df = extract_datetime_features(df, 'idle_start_time_mdt', timezone=None)
df

,idle_start_time,idle_end_time,idle_duration_hr,idle_location_x,idle_location_y,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt,timestamp,idle_start_hour
0,2025-05-08 00:44:15+00:00,2025-05-08 00:55:48+00:00,0.192500,-1140674450,509021916,2025-05-07 18:44:15-06:00,2025-05-07 18:55:48-06:00,18.916667,2025-05-07 18:44:15-06:00,18
1,2025-05-08 01:06:07+00:00,2025-05-08 02:03:30.010000+00:00,0.956392,-1140102166,508591116,2025-05-07 19:06:07-06:00,2025-05-07 20:03:30.010000-06:00,19.050000,2025-05-07 19:06:07-06:00,19
2,2025-05-08 02:21:39+00:00,2025-05-08 02:27:57.010000+00:00,0.105003,-1141278866,509446300,2025-05-07 20:21:39-06:00,2025-05-07 20:27:57.010000-06:00,20.450000,2025-05-07 20:21:39-06:00,20
3,2025-05-08 02:29:56.010000+00:00,2025-05-08 13:51:49+00:00,11.364719,-1141283733,509446566,2025-05-07 20:29:56.010000-06:00,2025-05-08 07:51:49-06:00,20.850000,2025-05-07 20:29:56.010000-06:00,20
4,2025-05-08 14:19:32.010000+00:00,2025-05-08 14:45:09+00:00,0.426942,-1141400783,510382683,2025-05-08 08:19:32.010000-06:00,2025-05-08 08:45:09-06:00,8.750000,2025-05-08 08:19:32.010000-06:00,8
...,...,...,...,...,...,...,...,...,...,...
313,2025-06-10 00:01:37+00:00,2025-06-10 01:13:04+00:00,1.190833,-1140877250,511574866,2025-06-09 18:01:37-06:00,2025-06-09 19:13:04-06:00,18.216667,2025-06-09 18:01:37-06:00,18
314,2025-06-10 01:51:55+00:00,2025-06-10 14:28:37+00:00,12.611667,-1141283433,509446050,2025-06-09 19:51:55-06:00,2025-06-10 08:28:37-06:00,19.466667,2025-06-09 19:51:55-06:00,19
315,2025-06-10 15:24:35+00:00,2025-06-10 16:20:23+00:00,0.930000,-1141091933,511507583,2025-06-10 09:24:35-06:00,2025-06-10 10:20:23-06:00,9.333333,2025-06-10 09:24:35-06:00,9
316,2025-06-10 16:22:11+00:00,2025-06-10 16:31:29+00:00,0.155000,-1141093166,511512416,2025-06-10 10:22:11-06:00,2025-06-10 10:31:29-06:00,10.516667,2025-06-10 10:22:11-06:00,10


In [6]:
# # Converting the idle start time for mdt time into a timestamp and extracting the hour from it
# df['timestamp'] = pd.to_datetime(df['idle_start_time_mdt'], format='mixed')
# df['idle_start_hour'] = df['timestamp'].dt.hour

In [7]:
df = add_week_tracking(df, 'timestamp')
df

,idle_start_time,idle_end_time,idle_duration_hr,idle_location_x,idle_location_y,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt,timestamp,idle_start_hour,week_start_date,week_number,week_end_date
0,2025-05-08 00:44:15+00:00,2025-05-08 00:55:48+00:00,0.192500,-1140674450,509021916,2025-05-07 18:44:15-06:00,2025-05-07 18:55:48-06:00,18.916667,2025-05-07 18:44:15.000,18,2025-05-06,1,2025-05-12
1,2025-05-08 01:06:07+00:00,2025-05-08 02:03:30.010000+00:00,0.956392,-1140102166,508591116,2025-05-07 19:06:07-06:00,2025-05-07 20:03:30.010000-06:00,19.050000,2025-05-07 19:06:07.000,19,2025-05-06,1,2025-05-12
2,2025-05-08 02:21:39+00:00,2025-05-08 02:27:57.010000+00:00,0.105003,-1141278866,509446300,2025-05-07 20:21:39-06:00,2025-05-07 20:27:57.010000-06:00,20.450000,2025-05-07 20:21:39.000,20,2025-05-06,1,2025-05-12
3,2025-05-08 02:29:56.010000+00:00,2025-05-08 13:51:49+00:00,11.364719,-1141283733,509446566,2025-05-07 20:29:56.010000-06:00,2025-05-08 07:51:49-06:00,20.850000,2025-05-07 20:29:56.010,20,2025-05-06,1,2025-05-12
4,2025-05-08 14:19:32.010000+00:00,2025-05-08 14:45:09+00:00,0.426942,-1141400783,510382683,2025-05-08 08:19:32.010000-06:00,2025-05-08 08:45:09-06:00,8.750000,2025-05-08 08:19:32.010,8,2025-05-06,1,2025-05-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,2025-06-10 00:01:37+00:00,2025-06-10 01:13:04+00:00,1.190833,-1140877250,511574866,2025-06-09 18:01:37-06:00,2025-06-09 19:13:04-06:00,18.216667,2025-06-09 18:01:37.000,18,2025-06-03,5,2025-06-09
314,2025-06-10 01:51:55+00:00,2025-06-10 14:28:37+00:00,12.611667,-1141283433,509446050,2025-06-09 19:51:55-06:00,2025-06-10 08:28:37-06:00,19.466667,2025-06-09 19:51:55.000,19,2025-06-03,5,2025-06-09
315,2025-06-10 15:24:35+00:00,2025-06-10 16:20:23+00:00,0.930000,-1141091933,511507583,2025-06-10 09:24:35-06:00,2025-06-10 10:20:23-06:00,9.333333,2025-06-10 09:24:35.000,9,2025-06-10,6,2025-06-16
316,2025-06-10 16:22:11+00:00,2025-06-10 16:31:29+00:00,0.155000,-1141093166,511512416,2025-06-10 10:22:11-06:00,2025-06-10 10:31:29-06:00,10.516667,2025-06-10 10:22:11.000,10,2025-06-10,6,2025-06-16


In [8]:
# # Tracking the number of weeks
# # Remove timezone info 
# df['timestamp'] = df['timestamp'].dt.tz_localize(None)

# # Align to Monday-start weeks
# df['week_start_date'] = df['timestamp'].dt.to_period('W-MON').dt.start_time

# # Get the first week start
# first_week = df['week_start_date'].min()

# # Assign custom week numbers starting from 1
# df['week_number'] = ((df['week_start_date'] - first_week).dt.days // 7) + 1

# # Calculate week end (Sunday)
# df['week_end_date'] = df['week_start_date'] + pd.Timedelta(days=6)

In [9]:
df = convert_idle_location_to_coords(df, 'idle_location_x', 'idle_location_y', drop_original=True)
df

,idle_start_time,idle_end_time,idle_duration_hr,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt,timestamp,idle_start_hour,week_start_date,week_number,week_end_date,lat,lon
0,2025-05-08 00:44:15+00:00,2025-05-08 00:55:48+00:00,0.192500,2025-05-07 18:44:15-06:00,2025-05-07 18:55:48-06:00,18.916667,2025-05-07 18:44:15.000,18,2025-05-06,1,2025-05-12,50.902192,-114.067445
1,2025-05-08 01:06:07+00:00,2025-05-08 02:03:30.010000+00:00,0.956392,2025-05-07 19:06:07-06:00,2025-05-07 20:03:30.010000-06:00,19.050000,2025-05-07 19:06:07.000,19,2025-05-06,1,2025-05-12,50.859112,-114.010217
2,2025-05-08 02:21:39+00:00,2025-05-08 02:27:57.010000+00:00,0.105003,2025-05-07 20:21:39-06:00,2025-05-07 20:27:57.010000-06:00,20.450000,2025-05-07 20:21:39.000,20,2025-05-06,1,2025-05-12,50.944630,-114.127887
3,2025-05-08 02:29:56.010000+00:00,2025-05-08 13:51:49+00:00,11.364719,2025-05-07 20:29:56.010000-06:00,2025-05-08 07:51:49-06:00,20.850000,2025-05-07 20:29:56.010,20,2025-05-06,1,2025-05-12,50.944657,-114.128373
4,2025-05-08 14:19:32.010000+00:00,2025-05-08 14:45:09+00:00,0.426942,2025-05-08 08:19:32.010000-06:00,2025-05-08 08:45:09-06:00,8.750000,2025-05-08 08:19:32.010,8,2025-05-06,1,2025-05-12,51.038268,-114.140078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,2025-06-10 00:01:37+00:00,2025-06-10 01:13:04+00:00,1.190833,2025-06-09 18:01:37-06:00,2025-06-09 19:13:04-06:00,18.216667,2025-06-09 18:01:37.000,18,2025-06-03,5,2025-06-09,51.157487,-114.087725
314,2025-06-10 01:51:55+00:00,2025-06-10 14:28:37+00:00,12.611667,2025-06-09 19:51:55-06:00,2025-06-10 08:28:37-06:00,19.466667,2025-06-09 19:51:55.000,19,2025-06-03,5,2025-06-09,50.944605,-114.128343
315,2025-06-10 15:24:35+00:00,2025-06-10 16:20:23+00:00,0.930000,2025-06-10 09:24:35-06:00,2025-06-10 10:20:23-06:00,9.333333,2025-06-10 09:24:35.000,9,2025-06-10,6,2025-06-16,51.150758,-114.109193
316,2025-06-10 16:22:11+00:00,2025-06-10 16:31:29+00:00,0.155000,2025-06-10 10:22:11-06:00,2025-06-10 10:31:29-06:00,10.516667,2025-06-10 10:22:11.000,10,2025-06-10,6,2025-06-16,51.151242,-114.109317


In [10]:
# # Converting the idle locations into proper coordinate format
# df['lat'] = df['idle_location_y'] / 1e7
# df['lon'] = df['idle_location_x'] / 1e7

## Clustering and Visualization

In [12]:
# select features
features = select_idle_features(df, feature_cols=['lat', 'lon', 'idle_start_hour', 'idle_duration_hr', 'week_number'], latest_week_only=False)
features

,lat,lon,idle_start_hour,idle_duration_hr,week_number
0,50.902192,-114.067445,18,0.192500,1
1,50.859112,-114.010217,19,0.956392,1
2,50.944630,-114.127887,20,0.105003,1
3,50.944657,-114.128373,20,11.364719,1
4,51.038268,-114.140078,8,0.426942,1
...,...,...,...,...,...
313,51.157487,-114.087725,18,1.190833,5
314,50.944605,-114.128343,19,12.611667,5
315,51.150758,-114.109193,9,0.930000,6
316,51.151242,-114.109317,10,0.155000,6


In [13]:
# # select features
# features = df[['lat', 'lon', 'idle_start_hour', 'idle_duration_hr', 'week_number']].copy()
# latest_week = df['week_start_date'].max()
# features_latest_week = df[df['week_start_date'] == latest_week][['lat', 'lon', 'idle_start_hour', 'idle_duration_hr', 'week_number']]

In [14]:
features = cluster_downtime(features)
features

,lat,lon,idle_start_hour,idle_duration_hr,week_number,cluster
0,50.902192,-114.067445,18,0.192500,1,-1
1,50.859112,-114.010217,19,0.956392,1,-1
2,50.944630,-114.127887,20,0.105003,1,0
3,50.944657,-114.128373,20,11.364719,1,0
4,51.038268,-114.140078,8,0.426942,1,-1
...,...,...,...,...,...,...
313,51.157487,-114.087725,18,1.190833,5,-1
314,50.944605,-114.128343,19,12.611667,5,0
315,51.150758,-114.109193,9,0.930000,6,-1
316,51.151242,-114.109317,10,0.155000,6,-1


In [15]:
# # scale features
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(features)

In [16]:
# # Prepare coordinates
# coords = features[['lat', 'lon']].to_numpy()

In [17]:
# # dbscan, fit coordinates
# dbscan = DBSCAN(eps=0.004, min_samples=5).fit(coords)
# features.loc[:, 'cluster'] = dbscan.labels_

In [18]:
# checking the clusters in the features df
features['cluster'].value_counts()

cluster
-1    214
 0     71
 1     16
 2      9
 3      8
Name: count, dtype: int64

We can see that most of the datapoints are just noise (cluster -1), meaning that they arent natural downtime spots

In [20]:
cluster_summary = summarize_cluster_downtime(features, 'cluster', 'idle_duration_hr')
cluster_summary

,cluster,total_downtime,avg_downtime,frequency
0,0,621.875572,8.758811,71
1,1,1.475292,0.092206,16
2,2,0.596106,0.066234,9
3,3,2.044728,0.255591,8


In [21]:
# # Creating a df that ranks the cluster's natural downtime (average amount of time spent idle, 
# # how frequently the vehicle idles and the total idle time)
# cluster_summary = features[features['cluster'] != -1].groupby('cluster').agg({
#     'idle_duration_hr': ['sum', 'mean', 'count']
# }).reset_index()

# cluster_summary.columns = ['cluster', 'total_downtime', 'avg_downtime', 'frequency']
# cluster_summary

We can now see the best clusters based on their total downtime.

In [23]:
best_cluster = get_best_cluster(cluster_summary)
best_cluster

cluster             0.000000
total_downtime    621.875572
avg_downtime        8.758811
frequency          71.000000
Name: 0, dtype: float64

In [24]:
# best_cluster = cluster_summary.sort_values('total_downtime', ascending=False).iloc[0]
# print(best_cluster)

# second_best_cluster = cluster_summary.sort_values('total_downtime', ascending=False).iloc[1]
# print(second_best_cluster)

# third_best_cluster = cluster_summary.sort_values('total_downtime', ascending=False).iloc[2]
# print(third_best_cluster)

In [25]:
best_coords = get_best_cluster_coords(features, best_cluster, 'cluster', 'lat', 'lon')
best_coords

lat     50.944647
lon   -114.128327
dtype: float64

In [26]:
# # Identifying the best coordinates for the best cluster
# best_cluster_id = best_cluster['cluster']
# best_coords = features[features['cluster'] == best_cluster_id][['lat', 'lon']].mean()
# best_coords

In [27]:
# # Identifying best times based on the best vehicle downtime cluster
# best_times = features[features['cluster'] == best_cluster_id]['idle_start_hour'].value_counts().sort_index()
# best_times

In [28]:
# Show all clusters with duration as popup label
plot_idle_clusters_on_map(features, label_cols=['idle_duration_hr'])

In [29]:
# Show only the best cluster, with start hour and duration
best_cluster_id = best_cluster['cluster']
plot_idle_clusters_on_map(features, only_cluster=best_cluster_id, label_cols=['idle_start_hour', 'idle_duration_hr'])

In [30]:
# Map with no legend
plot_idle_clusters_on_map(features, show_legend=False)

In [31]:
# Plot all clusters from the latest week with popups
plot_idle_clusters_on_map(
    df=features,
    label_cols=['idle_start_hour', 'idle_duration_hr'],
    latest_week_only=True
)

In [32]:
# map_center = [features['lat'].mean(), features['lon'].mean()]

# m = folium.Map(location=map_center, zoom_start=12)
# colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue']

# # Plot points for the best cluster
# for _, row in features[features['cluster'] == features['cluster']].iterrows():
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=3,
#         color=colors[int(row['cluster']) % len(colors)] if row['cluster'] != -1 else 'gray',
#         fill=True,
#         fill_opacity=0.5
#     ).add_to(m)

# m  # Display the map

## Building a Reccommendation Model/Engine

In [34]:
recs = generate_service_booking_recommendation_text(features, include_latest_week=False)
recs

📊 Recommendation based on *overall data*:
- 📍 **Location:** Near latitude 50.94465, longitude -114.12833
- ⏰ **Best time to book service:** Around **14:00**
- 🔁 **Idle occurrences at this time:** 11
- 🕒 **Expected idle duration:** Approximately **8.76 hours**



In [70]:
# save features df
features.to_csv('features.csv', index=False)

### 1 Week Info

In [26]:
# select features
features_week_1 = features[features['week_number'] == 6][['lat', 'lon', 'idle_start_hour', 'idle_duration_hr']].copy()

features_week_1

,lat,lon,idle_start_hour,idle_duration_hr
315,51.150758,-114.109193,9,0.930000
316,51.151242,-114.109317,10,0.155000
317,51.170268,-114.109115,10,1.753056


In [28]:
# scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_week_1)

In [30]:
# Prepare coordinates
coords = features_week_1[['lat', 'lon']].to_numpy()

In [32]:
# dbscan, fit coordinates
dbscan = DBSCAN(eps=0.004, min_samples=5).fit(coords)
features_week_1.loc[:, 'cluster'] = dbscan.labels_

In [33]:
# checking the clusters in the features df
features_week_1['cluster'].value_counts()

cluster
-1    3
Name: count, dtype: int64

In [36]:
# Creating a df that ranks the cluster's natural downtime (average amount of time spent idle, 
# how frequently the vehicle idles and the total idle time)
cluster_summary_week_1 = features_week_1[features_week_1['cluster'] != -1].groupby('cluster').agg({
    'idle_duration_hr': ['sum', 'mean', 'count']
}).reset_index()

cluster_summary_week_1.columns = ['cluster', 'total_downtime', 'avg_downtime', 'frequency']
cluster_summary_week_1

,cluster,total_downtime,avg_downtime,frequency


In [38]:
best_cluster_week_1 = cluster_summary_week_1.sort_values('total_downtime', ascending=False).iloc[0]
best_cluster_week_1

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Identifying the best coordinates for the best cluster
best_cluster_id_week_1 = best_cluster_week_1['cluster']
best_coords_week_1 = features_week_1[features_week_1['cluster'] == best_cluster_id][['lat', 'lon']].mean()

In [ ]:
print(best_coords_week_1)
print(best_coords)

In [ ]:
# Identifying best times based on the best vehicle downtime cluster
best_times_week_1 = features_week_1[features_week_1['cluster'] == best_cluster_id_week_1]['idle_start_hour'].value_counts().sort_index()
best_times_week_1

In [ ]:
map_center = [features_week_1['lat'].mean(), features_week_1['lon'].mean()]

m = folium.Map(location=map_center, zoom_start=12)
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'cadetblue']

# Plot points for the best cluster
for _, row in features_week_1[features_week_1['cluster'] == features_week_1['cluster']].iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        color=colors[int(row['cluster']) % len(colors)] if row['cluster'] != -1 else 'gray',
        fill=True,
        fill_opacity=0.5
    ).add_to(m)

m  # Display the map